# Using Greenland Ice Mapping Project Data Tools + ICESat-2 Data
---

This notebook illustrates some of the various tools and capabilities available to access and explore imagery and velocity data products from the Greenland Ice Mapping Project [GrIMP](https://nsidc.org/grimp) for further details.. Specifically, we will use the functionality of `nisarVel` and `nisarVelSeries` classes for working with GrIMP velocity products.

This notebook will also include examples of how to quickly explore imagery and velocity mosaics, formmatted as cloud optimized geotiffs (more on that below) to interactively select and download subsets of GrIMP image (NSIDC-0723) and velocity (NSIDC-481, 0725, 0727, 0731, 0766) data. For the Sentinel-based velocity mosaics (0725, 0727, 0731), a user can select a box on a map and choose which components are downloaded (vv, vx, vy, ex, ey, dT) and saved to a netCDF file. 

We will explore some of this functionality, read in and plot velocities over an area of interest, quickly run some statistical calculations to visualoize how ice flow has changed, and also evaluate these changes alongside ICESat-2 land ice elevation change. 

![GrIMP header](/home/jovyan/shared-public/ICESat-2-Hackweek/GrIMP/GrIMP_tutorial_res/GrIMP_header.png)

## Environment setup

The following packages are needed to execute this notebook. The notebook has been tested with the `environment.yml` in the *binder* folder of this repository. Thus, for best results, create a new conda environment to run this and other other GrIMP notebooks from this repository. 

`conda env create -f binder/environment.yml`

`conda activate greenlandMapping`

`python -m ipykernel install --user --name=greenlandMapping`

`jupyter lab`

See [NSIDCLoginNotebook](https://github.com/fastice/GrIMPNotebooks/blob/master/NSIDCLoginNotebook.ipynb) for additional information.

The notebooks can be run on a temporary virtial instance (to start click [**binder**](https://mybinder.org/v2/gh/fastice/GrIMPNotebooks/HEAD?urlpath=lab)). See the github [README](https://github.com/fastice/GrIMPNotebooks#readme) for further details.

```{note}
In order to use the full functionality of GrIMP notebooks for this tutorial, we will pip install two
two packages with funcitons for reading, subsetting, plotting, and downloading various datasets.
```

In [ ]:
%pip install git+https://github.com/fastice/grimpfunc.git@master
%pip install git+https://github.com/fastice/nisardev.git@main

Now for the normal imports

In [ ]:
import ipywidgets

import pandas as pd
#import geopandas as gpd
import json

import IS2view
import icepyx as ipx
import h5py
import warnings
import pyproj
import numpy as np
import earthaccess
import glob
import os
import sys
import re
import matplotlib.pyplot as plt
from IPython.display import Image
import rasterio
from rasterio import plot


import dask
from dask.diagnostics import ProgressBar
ProgressBar().register()
dask.config.set(num_workers=2)
import nisardev as nisar
import matplotlib.colors as mcolors
import grimpfunc as grimp
import panel
panel.extension()
from datetime import datetime
import glob
import xarray as xr
from shapely.geometry import box, Polygon

import rioxarray #as rio # Used to read raster data from hdf5 files

from IPython.lib.deepreload import reload

# Ignore warning
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

In [ ]:
# For searching NASA data
auth = earthaccess.login()

## More tutorial guidance:
See our [YouTube](https://www.youtube.com/@GreenlandIceMappingProject) page for more:

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0kIcSXkxaxI" \
     frameborder="0" allow="accelerometer" allowfullscreen></iframe>')

```{note}
**To get help and see options for any of the GrIMP or other functions while the cursor is \
positioned inside a method's parentheses, click shift+Tab.**
```

GrIMP velocity products are as stored at NSIDC in cloud-optimized geotif (COG) format with each compenent stored as separate band (e.g., vx, vy). In this notebook, we focus on on the velocity data, but the error and compenents can be similarly processed.

For reading the data, the products are specified with a single root file name (e.g., for *filename.vx(vy).othertext.tif*). For example, the version 3 annual mosaic December 2017 to November 2018 is specified as `GL_vel_mosaic_Annual_01Dec17_30Nov18_*_v03.0`. For locally stored files, the corresponding path to the data must be provided. For remote data, the https link is required. 

In [ ]:
myUrls = grimp.cmrUrls(mode='subsetter')  # Subsetter mode is required for subsetting.
display(myUrls.initialSearch())

This cell will pop up a search tool for the GrIMP products, which will run a predefined search for the annual products. While in principle, other products (e.g., six-day to quarterly can be retrieved, the rest of the notebook will need some modifications to accomodate).

In [ ]:
myVelSeries = nisar.nisarVelSeries() 
myVelSeries.readSeriesFromTiff(myUrls.getCogs(replace='vv', removeTiff=True), url=True, readSpeed=False, useErrors=True, useDT=True)  
myVelSeries.xr

## Preload Data and Select Bands
The cells in this section read the cloud-optimized geotiffs (COG) headers and create nisarVelSeries or nisarImageSeries objects for velocity or image data, respectively. The actual data are not downloaded at this stage, but the xarray internal to each object will read the header data of each product so it can efficiently access the data during later downloads. The bands (e.g., vx, vy) can be selected at this stage.

More detail can found on working with these tools in the workingWithGrIMPVelocityData and workingWithGrIMPImageData notebooks. 

```{note}
The  nisar class is used to read, display, velocity maps with a single time stamp. In this example, the `readSpeed=False` (default) forces the speed to be calculated from the individual components rather than read from a file, which is much quicker.

: Here is an example of how to access header information remotely and see the structure and total size of your potential download

:     myVelSeries = nisar.nisarVelSeries() 
      myVelSeries.readSeriesFromTiff(myUrls.getCogs(replace='vv', removeTiff=True), url=True,      readSpeed=False, useErrors=True, useDT=True)  
      myVelSeries.xr
```

## For now, let's use a local files to see examples of these returns

In [ ]:
myVelSeries = nisar.nisarVelSeries() # Initialize a nisarVelSeries instance:
localUrl = "/home/jovyan/shared-public/ICESat-2-Hackweek/GrIMP/GrIMP_tutorial_res/Geotiffs/GL_vel_mosaic_Annual_01Dec20_30Nov21_*_v04.0"
myVelSeries.readSeriesFromTiff([localUrl], useDT=False,useErrors=False,readSpeed=True,url=False,useStack=False,overviewLevel=0)  
myVelSeries.xr

## Examining interesting velocity map features

The image below is an example of an annual GrIMP velocity mosaic, withfilename `GL_vel_mosaic_Annual_01Dec17_30Nov18_*_v03.0` (accessed at [**here**](https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0727.005/2022.09.01/))

![GrIMP mosaic](/home/jovyan/shared-public/ICESat-2-Hackweek/GrIMP/GrIMP_tutorial_res/Geotiffs/GL_vel_mosaic_Quarterly_01Sep22_30Nov22_browse_v05.0.jpg)

## **These files can be large!**

Below are functions that allow you to quickly subset the data by spatial bounds, including the functions you can use to interactively select an ROI when exploring datasets remotely

## Method 1: Interactive ROI Selection
Run the next the tool below to select the bounding box (or modify a manually selected box), which will display a SAR image map. Depending on network speed, it could take a few seconds to a minute to load the basemap. Use the box tool in the plot menu to select a region of interest. 

```{admonition} Interactive ROI selection
:
:     if 'boxPicker' not in locals(): # Only create if not defined above
          boxPicker = grimp.boxPicker()
      boxPicker.plotMap(show=(not myUrls.checkIDs(['NSIDC-0481']) and not myUrls.checkIDs(['NSIDC-0646'])))  # Skips map if a 481 product
```

## Method 2: Manual Selection
The coordinates for bounding box, bbox, can be manually entered by modifying the cell below with the desired values. Even if not using interactive selection, running that step displays the manually selected box coordinates on radar map of Greenland. Note by default, coordinates are rounded to the nearest kilometer. We will a use predefined bbox for the tutorial today.

In [ ]:
values = np.around([-472889, -2540695, -453487, -2523127])
xyBounds = dict(zip(['minx', 'miny', 'maxx', 'maxy'], values))
xyBounds

myVelSeries.subSetVel(xyBounds)
myVelSeries.subset

The data in the above example map are stored as an Xarray, velMap.subset. In this case, the subset is the full map of Greenland over several timesteps, which is > 8GB and could take a while to download! Because of the lazy open mentioned above, the data have not been downloaded or read from disk yet. Before applying the final subset, its useful to examine the size of the full data (virtual) array. If the loadDataArray step was sucessful, this next cell will provide details on the size and organization of the full xarray (prior to any download).

```{admonition} Optionally download subset as netcdf file
The downloaded subset can be saved in a netcdf and reloaded for to velSeries instance for later   analysis.   Note if the data have been subsetted, ONLY the subset will be saved - so it is not a bad idea to check out the dimensions, variable names, and total size of the subset data one last time prior to downloading.

: Add comment on xy bounds           
:     myVelSeries.subSetVel(xyBounds)
      myVelSeries.subset

: Add comment on export to cdf
:     subsetFile = 'steenstrupGlacier_annVel.nc'
      myVelSeries.loadRemote()
      myVelSeries.toNetCDF(subsetFile)
      
```


## For today, however, we are going to assume we have already subset and downloaded our area and variable of interest, and read in velocities over Steenstrup Glacier, Greenland.

In [ ]:
# Now reload the downloaded data
myVelReload = nisar.nisarVelSeries() #shared-public/ICESat-2-Hackweek/
myVelReload.readSeriesFromNetCDF('/home/jovyan/shared-public/ICESat-2-Hackweek/GrIMP/GrIMP_tutorial_res/steenstrupGlacier_annVel.nc')
myVelReload.loadRemote()

In [ ]:
myVelReload.xr #confirm dimensions of read-in subsetted velocities

## We can plot the annual velocity maps for Steenstrup Glacier and explore multiyear trends using an inspection tools in the cell blocks below

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(18,18))

for ax, date in zip(axes.flatten(), myVelReload.time): 
    myVelReload.displayVelForDate(date=date, ax=ax,units='km')
axes[-1, -1].axis('off'); #remove any empty axes if odd number of years
fig.suptitle('Time Series of Speed', fontsize=18)
fig.tight_layout()

In [ ]:
myVelReload.inspect()

## Quickly plot velocity anomalies
computes [velocity at t_i - mean] of all timesteps in data series

In [ ]:
velAnomaly = myVelReload.anomaly()

# Plot the anomaly for each year
fig, axes = plt.subplots(2, 4, figsize=(20,10))
for date, ax in zip(velAnomaly.time, axes.flatten()):
    velAnomaly.displayVelForDate(date, ax=ax, units='m', vmin=-600, vmax=600, autoScale=False, 
                                 cmap='RdBu_r', colorBarLabel='Speed Anomaly (m/yr)', extend='both')
axes[-1, -1].axis('off'); # kill axis with no plot
fig.suptitle('Speed Anomalies by Year', fontsize=16)
fig.tight_layout()

## Dynamic Thinning at ice sheet margins
Typically, we expect a rapid acceleration in ice flow speeds to also correspond to ice thinning. Let's use ICESat-2 data to see if that is true for Steenstrup Glacier.

![glacier thinning](/home/jovyan/shared-public/ICESat-2-Hackweek//GrIMP/GrIMP_tutorial_res/dynamic_thinning_schematic.png)

```{admonition} Accessing from the cloud (outside of tutorial)
Follow the example block of code below to access non-local data outside of tutorial  

: Give ROI, product, and temporal range to query using icepyx
:     spatial_extent = [-34.8799,66.4665,-34.3847,66.6506]
      region_a = ipx.Query('ATL06',spatial_extent,['2019-01-01','2021-12-31'])
      region_a.earthdata_login()
      
: Explore variables, select those of interest and download submitted granule order.
:     region_a.show_custom_options(dictview=True)
      region_a.order_vars.avail(options=True)
      region_a.visualize_spatial_extent()
      region_a.download_granules('/home/jovyan/icesat2data') #download path (replace with preferred directory)
```

## Read in local ATL06 data 


## Use Ben Smith's function to read in ATL06 (ICESat-2 land ice product) data from local files

In [ ]:
def atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=None):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            beam: a string specifying which beam is to be read (ex: gt1l, gt1r, gt2l, etc)
            field_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
            index: which entries in each field to read
            epsg: an EPSG code specifying a projection (see www.epsg.org).  Good choices are:
                for Greenland, 3413 (polar stereographic projection, with Greenland along the Y axis)
                for Antarctica, 3031 (polar stereographic projection, centered on the Pouth Pole)
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a numpy array containing the 
                data
    """
    if field_dict is None:
        field_dict={None:['latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}
    D={}
    file_re=re.compile('ATL06_(?P<date>\d+)_(?P<rgt>\d\d\d\d)(?P<cycle>\d\d)(?P<region>\d\d)_(?P<release>\d\d\d)_(?P<version>\d\d).h5')
    with h5py.File(filename,'r') as h5f:
        for key in field_dict:
            for ds in field_dict[key]:
                if key is not None:
                    ds_name=beam+'/land_ice_segments/'+key+'/'+ds
                else:
                    ds_name=beam+'/land_ice_segments/'+ds
                if index is not None:
                    D[ds]=np.array(h5f[ds_name][index])
                else:
                    D[ds]=np.array(h5f[ds_name])
                if '_FillValue' in h5f[ds_name].attrs:
                    bad_vals=D[ds]==h5f[ds_name].attrs['_FillValue']
                    D[ds]=D[ds].astype(float)
                    D[ds][bad_vals]=np.NaN
    if epsg is not None:
        xy=np.array(pyproj.proj.Proj(epsg)(D['longitude'], D['latitude']))
        D['x']=xy[0,:].reshape(D['latitude'].shape)
        D['y']=xy[1,:].reshape(D['latitude'].shape)
    temp=file_re.search(filename)
    D['rgt']=int(temp['rgt'])
    D['cycle']=int(temp['cycle'])
    D['beam']=beam
    return D

In [ ]:
# Read in local SAR mosaic of our glacier of interest

img_name = '/home/jovyan/GrIMP_tutorial_res/Geotiffs/GL_S1bks_mosaic_uncalibrated._2021-08-05.tif'
img = rasterio.open(img_name)

# read in first band of imagery
array = img.read(1)
array[array < 0] = 0

# Read ATL06 land ice elevation from GT2l track
ATL06_file0=('/home/jovyan/shared-public/ICESat-2-Hackweek/GrIMP/GrIMP_tutorial_res/icesat2data/processed_ATL06_20191106230008_06270503_006_01.h5')
D0=atl06_to_dict(ATL06_file0,'/gt2l', index=None, epsg=3413)
ATL06_file1=('/home/jovyan/shared-public/ICESat-2-Hackweek/GrIMP/GrIMP_tutorial_res/icesat2data/processed_ATL06_20211102121853_06271303_006_01.h5')
D1=atl06_to_dict(ATL06_file1,'/gt2l', index=None, epsg=3413)

## Now plot ICESat-2 and velocity change together to evaluate if dynamic thinning occurred

In [ ]:
fig = plt.figure(figsize=(28,10))
ax1 = plt.subplot(1,2,1)   
ax2 = plt.subplot(1,2,2)

rasterio.plot.show(array,ax=ax1,cmap='gist_gray',transform=img.transform,vmin = 10, vmax = 180)

#tmp vars for plotting
xCopy = D1['x']
yCopy = D1['y']


ax1.plot(D1['x'], D1['y'],'c.',label='ATL06 GTL02 track')
ax1.plot(xCopy[700:1000], yCopy[700:1000],'w.',label='track seg. over glacier')
ax1.set_xlim(450000,470000)
ax1.set_ylim(-2543000,-2523000)
ax1.legend();
ax1.tick_params(axis='both', labelsize=12)

ax2.plot(D0['x_atc'], D0['h_li'],'r.', label='ATL06_June_2019')
ax2.plot(D1['x_atc'], D1['h_li'],'b.', label='ATL06_June_2021')
ax2.set_xlim(holder[700], holder[1000])
ax2.set_ylim(500,750)
ax2.tick_params(axis='both', labelsize='large')
ax2.set_ylabel('Ice Elevation (m)',fontsize=20)
ax2.legend(fontsize=20); 